In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os

#prepara os dados
train_path = "../input/train/train"
test_path = "../input/test/test"

label_frame = pd.read_csv('../input/train.csv')
test_frame = pd.read_csv('../input/sample_submission.csv')
x_train = []
x_test = []
y_train = np.array(label_frame['has_cactus'])
#load images
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
for fname in label_frame['id']:
    image_path = os.path.join(train_path , fname)
    pil_image = image.load_img(image_path, target_size=(32, 32, 3))
    np_image = image.img_to_array(pil_image)
    x_train.append(np_image)
for fname in test_frame['id']:
    image_path = os.path.join(test_path,fname)
    pil_image = image.load_img(image_path,target_size = (32,32,3))
    np_image = image.img_to_array(pil_image)
    x_test.append(np_image)
#transfoma para array
x_train = np.array(x_train)
x_train = x_train.astype('float32')/255
x_test = np.array(x_test)
x_test = x_test.astype('float32')/255
print(x_train.shape)
print(x_test.shape)

# Overveiw sobre ConvNet

A convolução é uma parte da construção de uma rede neural convolucional. No campo da visão computacional, uma imagem pode ser expressada como uma matriz de valores RGB. Por isso, vamos considerar a matriz 6x6 abaixo como uma parte de uma imagem:
![](https://cdn-images-1.medium.com/max/800/1*aGSthcPASa2OT1UBm7paOA.png)
E o filtro será a seguinte matriz:
![](https://cdn-images-1.medium.com/max/800/1*591OPcvDKUN9liZ_VQ1M5g.png)
Então a convolução é a aplicação desse filtro na matriz da imagem, adicionando o produto dos valores do filtro e os valores da matriz da imagem, que nesse caso, vai gerar uma camada convolucionada 4x4.

Essa animação ajuda e explicar melhor a convolução:
![](https://cdn-images-1.medium.com/max/800/1*VJCoCYjnjBBtWDLrugCBYQ.gif)

Convoluções são definidas por dois parâmetros:
- O tamanho do filtro ou pacote que será extraído da variável de entrada. Nesse caso 3x3
- O número de filtros computados da convolução

### Maxpooling

Maxpooling consiste na extração de variáveis dos mapas de entrada, tirando os valores máximos de cada canal.
![](https://cdn-images-1.medium.com/max/800/1*Ziqq69FhwOAbBi9-FNruAA.png)

# Data augmentation

A performance de redes neurais na maioria das vezes é melhorada com a quantidade de dados disponível. Data augmentation é a técnica para artificialmente criar novos dados de treino a partir de dados existentes como o redimencionamento de imagens, por exemplo.

In [ ]:
augmentations = ImageDataGenerator(
    vertical_flip=True,
    horizontal_flip=True,
    zoom_range=0.1)

augmentations.fit(x_train)

# Construindo o modelo

Como a base de treino não é grande e as imagens da mesma não se diferem das encontrada na base imagenet, utilizaremos a transferência de aprendizado do modelo imagenet.
A abordagen da transferëncia será a utilização das informações levantadas. Com isso, a última camada será retirada e no lugar será colocada uma camada desnsa MLP com função de ativação sigmoidal, uma vez que se trata de um problema de classificação binária.
Foi utilizado o VGG16 para imagenet.
Além do imagenet, foram utilizadas:
1- camada de dropout como técnica de regularização utilizada com o intuito de reduzir o overfitting da rede durante o treino;
2- camada de Batch Normalization que normaliza os inputs das camadas escondidas, aumentando assim a velocidade de treino da rede neural;


In [ ]:
#build model
from keras.applications import VGG16
from keras import models
from keras import layers
from keras import optimizers
from keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau
def get_model():
    base = VGG16(include_top = False,weights = 'imagenet',input_shape = (32,32,3))
    base.trainable = True
    base.summary()
    set_trainable = False
    for layer in base.layers:
        if layer.name == 'block5_conv3':
            set_trainable = True
        if set_trainable:
            layer.trainable = True
        else:
            layer.trainable = False
    model = models.Sequential()
    model.add(base)
    model.add(layers.Flatten())
    
    model.add(layers.BatchNormalization())
    
    model.add(layers.Dense(256,activation = 'relu'))
    
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1,activation = 'sigmoid'))
    model.summary()
    model.compile(loss = 'binary_crossentropy',optimizer = 'adam',metrics = ['acc'])
    return model
    
model = get_model()


# Rodando o modelo, a predição e gravando arquivo de submissão

In [ ]:
model.fit_generator(augmentations.flow(x_train,y_train),epochs = 100, steps_per_epoch=150)
y_predictions = model.predict(x_test)
result = pd.DataFrame({'id' : pd.read_csv('../input/sample_submission.csv')['id'],'has_cactus' : y_predictions.squeeze()})
result.to_csv("submissionMax.csv", index=False, columns=['id', 'has_cactus'])
print('submit successful')